In [1]:
# Data PreProcessing

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the dataset directory and categories
DATADIR = "dataset"
CATEGORIES = ["Yes","No","Smile","Fist","Hello","I Like You","Call me","Okay","Peace"]

# Define the image size and batch size
IMG_SIZE = 128
BATCH_SIZE = 32

# Initialize the training and validation data arrays
train_data = []
val_data = []

# Iterate through the categories and preprocess the images
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        img_array = img_array.astype('float32') / 255.0
        if np.random.rand() < 0.8:
            train_data.append([img_array, class_num])
        else:
            val_data.append([img_array, class_num])

# Shuffle the training and validation data arrays
np.random.shuffle(train_data)
np.random.shuffle(val_data)

# Split the training and validation data arrays into features and labels
train_features = np.array([data[0] for data in train_data])
train_labels = np.array([data[1] for data in train_data])
val_features = np.array([data[0] for data in val_data])
val_labels = np.array([data[1] for data in val_data])

# Split the training data into training and validation sets
train_features, val_features, train_labels, val_labels = train_test_split(train_features, train_labels, test_size=0.2)

# Define the training and validation data generators with data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow(
    train_features, train_labels,
    batch_size=BATCH_SIZE
)

val_generator = val_datagen.flow(
    val_features, val_labels,
    batch_size=BATCH_SIZE
)


In [1]:
# Training the model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# Define the input shape and number of classes
INPUT_SHAPE = (128, 128, 3)
NUM_CLASSES = 10

# Define the model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model with appropriate optimizer, loss function, and metrics
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the number of epochs and batch size
NUM_EPOCHS = 10
BATCH_SIZE = 32

# Train the model on the training data using the data generator
history = model.fit(train_generator, epochs=NUM_EPOCHS, validation_data=val_generator, batch_size=BATCH_SIZE)


In [2]:
# Validation

In [ ]:
# Evaluate the model on the test data using the test data generator
test_loss, test_acc = model.evaluate(test_generator)

# Print the test accuracy
print('Test accuracy:', test_acc)


In [3]:
# handMarks -- handdetection

In [1]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Initialize video capture
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process image and get hand landmarks
        results = hands.process(image)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Display the resulting image
        cv2.imshow('Hand Detection', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
